# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
J.-U. Pott  ->  J.-U. Pott  |  ['J.-U. Pott']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 74 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2502.01724


extracting tarball to tmp_2502.01724...

 done.
Retrieving document from  https://arxiv.org/e-print/2502.01744


extracting tarball to tmp_2502.01744...

...

extracting tarball to tmp_2502.02239...

 done.
Retrieving document from  https://arxiv.org/e-print/2502.02381


extracting tarball to tmp_2502.02381...

 done.


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2502.02381/master.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: './auger_resources/acknowledgments' from 'tmp_2502.02381/auger_resources/acknowledgments.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: './auger_resources/jcappub_authorlist' from 'tmp_2502.02381/auger_resources/jcappub_authorlist.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' f

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.01744-b31b1b.svg)](https://arxiv.org/abs/2502.01744) | **JWST sighting of decameter main-belt asteroids and view on meteorite sources**  |
|| A. Y. Burdanov, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2025-02-05*|
|*Comments*| *Original version submitted to Nature in June 2024; see final accepted version at DOI: s41586-024-08480-z*|
|**Abstract**|            Asteroid discoveries are essential for planetary-defense efforts aiming to prevent impacts with Earth, including the more frequent megaton explosions from decameter impactors. While large asteroids ($\geq$100 km) have remained in the main belt since their formation, small asteroids are commonly transported to the near-Earth object (NEO) population. However, due to the lack of direct observational constraints, their size-frequency distribution--which informs our understanding of the NEOs and the delivery of meteorite samples to Earth--varies significantly among models. Here, we report 138 detections of the smallest asteroids ($\gtrapprox $10 m) ever observed in the main belt, which were enabled by JWST's infrared capabilities covering the asteroids' emission peaks and synthetic tracking techniques. Despite small orbital arcs, we constrain the objects' distances and phase angles using known asteroids as proxies, allowing us to derive sizes via radiometric techniques. Their size-frequency distribution exhibits a break at ${\sim}100$ m (debiased cumulative slopes of $q = -2.66\pm0.60$ and $-0.97\pm0.14$ for diameters smaller and larger than $\sim $100 m, respectively), suggestive of a population driven by collisional cascade. These asteroids were sampled from multiple asteroid families--most likely Nysa, Polana and Massalia--according to the geometry of pointings considered here. Through additional long-stare infrared observations, JWST is poised to serendipitously detect thousands of decameter-scale asteroids across the sky, probing individual asteroid families and the source regions of meteorites "in-situ".         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.01840-b31b1b.svg)](https://arxiv.org/abs/2502.01840) | **Direct imaging of AGN outflows and their origin with the 23 m Large Binocular Telescope**  |
|| J. W. Isbell, et al. -- incl., <mark>J.-U. Pott</mark> |
|*Appeared on*| *2025-02-05*|
|*Comments*| *26 Pages, Accepted for publication in Nature Astronomy*|
|**Abstract**|            Active galactic nuclei (AGNs) are a key component of galaxy evolution due to feedback on the host from its supermassive black hole. The morphology of warm, in- and outflowing dusty material can reveal the nature of the onset of feedback, AGN feeding, and the unified model of AGN. Here we use the Large Binocular Telescope Interferometer (LBTI) to image the dense, obscuring disk and extended dusty outflow region of NGC 1068. In Fizeau imaging mode the LBTI synthesizes the equivalent resolution of a 22.8 m telescope. The 8.7 $\mu$m Fizeau images of NGC 1068 {have an effective resolution of $47\times90$ mas ($3.3\times6.2$ pc)} in a 5" field of view after performing PSF deconvolution techniques described here. This is the only extragalactic source to be Fizeau imaged using the LBTI, and the images bridge the scales measured with the Very Large Telescope Interferometer (VLTI; 0.5-5 pc) and those of single telescopes such as JWST and Keck ($>15$ pc). The images detect and spatially resolve the low surface brightness mid-infrared (MIR) features in the AGN disk/wind region that are over-resolved by the VLTI. The images show strong correlation between MIR dust emission and near-infrared (NIR) emission of highly excited atomic lines observed by SINFONI. Such LBTI imaging is a precursor to infrared imaging using the upcoming generation of extremely large telescopes, with angular resolutions up to 6x better than JWST, the largest space telescope in orbit.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.02381-b31b1b.svg)](https://arxiv.org/abs/2502.02381) | **Search for a diffuse flux of photons with energies above tens of PeV at the Pierre Auger Observatory**  |
|| P. A. Collaboration, et al. |
|*Appeared on*| *2025-02-05*|
|*Comments*| *Submitted to JCAP*|
|**Abstract**|            Diffuse photons of energy above 0.1 PeV, produced through the interactions between cosmic rays and either interstellar matter or background radiation fields, are powerful tracers of the distribution of cosmic rays in the Galaxy. Furthermore, the measurement of a diffuse photon flux would be an important probe to test models of super-heavy dark matter decaying into gamma-rays. In this work, we search for a diffuse photon flux in the energy range between 50 PeV and 200 PeV using data from the Pierre Auger Observatory. For the first time, we combine the air-shower measurements from a 2 km$^2$ surface array consisting of 19 water-Cherenkov surface detectors, spaced at 433 m, with the muon measurements from an array of buried scintillators placed in the same area. Using 15 months of data, collected while the array was still under construction, we derive upper limits to the integral photon flux ranging from 13.3 to 13.8 km$^{-2}$ sr$^{-1}$ yr$^{-1}$ above tens of PeV. We extend the Pierre Auger Observatory photon search program towards lower energies, covering more than three decades of cosmic-ray energy. This work lays the foundation for future diffuse photon searches: with the data from the next 10 years of operation of the Observatory, this limit is expected to improve by a factor of $\sim$20.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.01724-b31b1b.svg)](https://arxiv.org/abs/2502.01724) | **Cosmic quenching**  |
|| G. D. Lucia, F. Fontanot, M. Hirschmann, <mark>L. Xie</mark> |
|*Appeared on*| *2025-02-05*|
|*Comments*| *This is a pre-print of a chapter for the Encyclopedia of Astrophysics (edited by I. Mandel, section editor S. McGee) to be published by Elsevier as a Reference Module*|
|**Abstract**|            The quenching of star formation activity represents a critical phase for a non-negligible fraction of the observed galaxy population at all cosmic epochs, marking a transition from an epoch of intense mass growth to an extended period of passive evolution. Over the past years, we have collected a detailed characterization of how the fraction of quenched galaxies evolves as a function of cosmic time (it grows at later cosmic epochs), and correlates with the physical properties of galaxies (more massive galaxies, that also tend to be bulge-dominated, are more likely to be quenched) and with their environment (denser environments host larger fractions of quiescent galaxies). Different physical processes can lead to a suppression of star formation. These include internal processes (processes that do not depend on the environment in which galaxies live, e.g. stellar and AGN feedback mechanisms) and processes that instead depend on the environment (e.g. mergers, gas stripping processes). In this chapter, we summarize the observational and theoretical status of the field, highlighting the most recent results and the questions that are yet to be answered. We also summarize our view on the expected developments in the next few years.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.02239-b31b1b.svg)](https://arxiv.org/abs/2502.02239) | **The SRG/eROSITA all-sky survey: The morphologies of clusters of galaxies I: A catalogue of morphological parameters**  |
|| J. S. Sanders, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-02-05*|
|*Comments*| *Accepted by A&A, 32 pages, 24 figures*|
|**Abstract**|            The first SRG/eROSITA all-sky X-ray survey, eRASS1, resulted in a catalogue of over twelve thousand optically-confirmed galaxy groups and clusters in the western Galactic hemisphere. Using the eROSITA images of these objects, we measure and study their morphological properties, including their concentration, central density and slope, ellipticity, power ratios, photon asymmetry, centroid shift and Gini coefficient. We also introduce new forward-modelled parameters which take account of the instrument point spread function (PSF), which are slosh, which measures how asymmetric the surface brightness distribution is, and multipole magnitudes, which are analogues to power ratios. Using simulations, we find some non forward-modelled parameters are strongly biased due to PSF and data quality. For the same clusters, we find similar values of concentration and central density compared to results by ourselves using Chandra and previous results from XMM-Newton. The population as a whole has log concentrations which are typically around 0.3 dex larger than South Pole Telescope or Planck-selected samples and the deeper eFEDS sample. The exposure time, detection likelihood threshold, extension likelihood threshold and number of counts affect the concentration distribution, but generally not enough to reduce the concentration to match the other samples. The concentration of clusters in the survey strongly affects whether they are detected as a function of redshift and luminosity. We introduce a combined disturbance score based on a Gaussian mixture model fit to several of the parameters. For brighter clusters, around 1/4 of objects are classified as disturbed using this score, which may be due to our sensitivity to concentrated objects.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures []
exported in  _build/html/2502.01744.md
found figures []
exported in  _build/html/2502.01840.md


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\aj}{Astron. J.}$
$\newcommand{\actaa}{Acta Astronomica}$
$\newcommand{\araa}{Annual Review of Astronomy and Astrophysics}$
$\newcommand{\apj}{Astrophys. J.}$
$\newcommand{\apjl}{Astrophys. J., Letters}$
$\newcommand{\apjs}{Astrophys. J., Suppl. Ser.}$
$\newcommand{\ao}{Applied Optics}$
$\newcommand{\apss}{Astrophysics and Space Science}$
$\newcommand{\aap}{Astron. Astrophys.}$
$\newcommand{\aapr}{Astron. Astrophys. Reviews}$
$\newcommand{\aaps}{Astron. Astrophys., Suppl. Ser.}$
$\newcommand{\azh}{Astronomicheskii Zhurnal}$
$\newcommand{\baas}{Bulletin of the AAS}$
$\newcommand{\caa}{Chinese Astronomy and Astrophysics}$
$\newcommand{\cjaa}{Chinese Journal of Astronomy and Astrophysics}$
$\newcommand{\icarus}{Icarus}$
$\newcommand{\jcap}{Journal of Cosmology and Astroparticle Physics}$
$\newcommand{\jrasc}{Journal of the Royal Astronomical Society of Canada}$
$\newcommand{\memras}{Memoirs of the Royal Astronomical Society}$
$\newcommand{\mnras}{Mon. Not. R. Astron. Soc.}$
$\newcommand{\na}{New Astronomy}$
$\newcommand{\nar}{New Astronomy Review}$
$\newcommand{\pra}{Physical Review A: General Physics}$
$\newcommand{\prb}{Physical Review B: Solid State}$
$\newcommand{\prc}{Physical Review C}$
$\newcommand{\prd}{Physical Review D}$
$\newcommand{\pre}{Physical Review E}$
$\newcommand{\prl}{Physical Review Letters}$
$\newcommand{\pasa}{Publications of the Astron. Soc. of Australia}$
$\newcommand{\pasp}{Publ. Astron. Soc. Pacific}$
$\newcommand{\pasj}{Publications of the Astronomical Society of Japan}$
$\newcommand{\pnas}{Proc. Natl. Acad. Sci.}$
$\newcommand{\rmxaa}{Revista Mexicana de Astronomia y Astrofisica}$
$\newcommand{\qjras}{Quarterly Journal of the RAS}$
$\newcommand{\skytel}{Sky and Telescope}$
$\newcommand{\solphys}{Solar Physics}$
$\newcommand{\sovast}{Soviet Astronomy}$
$\newcommand{\ssr}{Space Science Reviews}$
$\newcommand{\zap}{Zeitschrift fuer Astrophysik}$
$\newcommand{\nat}{Nature}$
$\newcommand{\iaucirc}{IAU Cirulars}$
$\newcommand{\aplett}{Astrophysics Letters}$
$\newcommand{\apspr}{Astrophysics Space Physics Research}$
$\newcommand{\bain}{Bulletin Astronomical Institute of the Netherlands}$
$\newcommand{\fcp}{Fundamental Cosmic Physics}$
$\newcommand{\gca}{Geochimica Cosmochimica Acta}$
$\newcommand{\grl}{Geophysics Research Letters}$
$\newcommand{\jcp}{Journal of Chemical Physics}$
$\newcommand{\jgr}{Journal of Geophysics Research}$
$\newcommand{\jqsrt}{Journal of Quantitiative Spectroscopy and Radiative Transfer}$
$\newcommand{\memsai}{Mem. Societa Astronomica Italiana}$
$\newcommand{\nphysa}{Nuclear Physics A}$
$\newcommand{\physrep}{Physics Reports}$
$\newcommand{\physscr}{Physica Scripta}$
$\newcommand{\planss}{Planetary Space Science}$
$\newcommand{\procspie}{Proc. SPIE}$
$\newcommand{\maps}{Meteor. Planet. Sci.}$
$\newcommand\mb{#1}$
$\newcommand\tablefoot{#1}$</div>



<div id="title">

# $\red{Original version submitted in June 2024, see final version at DOI: s41586-024-08480-z.\ \\}$JWST sighting of decameter main-belt asteroids and view on meteorite sources

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.01744-b31b1b.svg)](https://arxiv.org/abs/2502.01744)<mark>Appeared on: 2025-02-05</mark> -  _Original version submitted to Nature in June 2024; see final accepted version at DOI: s41586-024-08480-z_

</div>
<div id="authors">

A. Y. Burdanov, et al. -- incl., <mark>L. Kreidberg</mark>

</div>
<div id="abstract">

**Abstract:** Asteroid discoveries are essential for planetary-defense efforts aiming to prevent impacts with Earth Cheng_2018P&SS..157..104C ,including the more frequent Brown2002 megaton explosionsfrom decameter impactors Chyba_1993Natur.361...40C, Chao_1960Sci...132..220C, Stoffler_2002M&PS...37.1893S, Reddy2019 .While large asteroids ( $\geq$ 100 km) have remained in the main belt since their formation Bottke_2005Icar..175..111B , small asteroidsare commonly transported to the near-Earth object (NEO) population Farinella_1998Icar..132..378F, Chesley_2003Sci...302.1739C .However, due to the lack of direct observational constraints,their size-frequency distribution---which informs our understanding of the NEOs and the delivery of meteorite samples to Earth---varies significantly among models Brown_2013Natur.503..238B, Harris_2021Icar..36514452H, Nesvorny_2024Icar..41115922N, Broz_2023, Marsset_2023 .Here, we report 139 detections of the smallest asteroids ( $\gtrapprox $ 10 m)ever observed in the main belt, which were enabled by JWST's infrared capabilitiescovering the asteroids' emission peaks Mueller2023A&A and synthetic tracking techniques tyson1992limits, shao2014finding, Burdanov2023MNRAS .Their size-frequency distribution exhibits a break at ${\sim}100 {\rm m}$ (the debiased cumulative slopes $q = -2.25$ and $-0.98$ , with the uncertainty ${\sim}0.1$ ),suggestive of a population driven by collisional cascade.These asteroids were sampled from multiple asteroid families---most likely Nysa, Polana and Massalia---according to the geometry of pointings considered here.Through additional long-stare infrared observations,JWST is poised to serendipitously detect thousands of decameter-scale asteroids across the sky,probing individual asteroid families Nesvorny_2015aste.book..297N and the source regions of meteorites Broz_2023, Marsset_2023 "in-situ".

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.01744"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\code}[1]{{\color{blue} \texttt{#1}}}$
$\newcommand{\pack}[1]{{\color{magenta} \texttt{#1}}}$
$\newcommand{\oiii}{\lbrack OIII\rbrack}$
$\newcommand{\actaa}{Acta Astron.}$
$\newcommand{\araa}{Annu. Rev. Astron. Astrophys.}$
$\newcommand{\areps}{Annu. Rev. Earth Planet. Sci.}$
$\newcommand{\aar}{Astron. Astrophys. Rev.}$
$\newcommand{\ab}{Astrobiology}$
$\newcommand{\aj}{Astron. J.}$
$\newcommand{\ac}{Astron. Comput.}$
$\newcommand{\apart}{Astropart. Phys.}$
$\newcommand{\apjl}{Astrophys. J. Lett.}$
$\newcommand{\apjs}{Astrophys. J. Suppl. Ser.}$
$\newcommand{\apss}{Astrophys. Space Sci.}$
$\newcommand{\aap}{Astron. Astrophys.}$
$\newcommand{\aapr}{Astron. Astrophys. Rev.}$
$\newcommand{\aaps}{Astron. Astrophys. Suppl.}$
$\newcommand{\baas}{Bull. Am. Astron. Soc.}$
$\newcommand{\caa}{Chin. Astron. Astrophys.}$
$\newcommand{\cjaa}{Chin. J. Astron. Astrophys.}$
$\newcommand{\cqg}{Class. Quantum Gravity}$
$\newcommand{\epsl}{Earth Planet. Sci. Lett.}$
$\newcommand{\expa}{Exp. Astron.}$
$\newcommand{\frass}{Front. Astron. Space Sci.}$
$\newcommand{\gal}{Galaxies}$
$\newcommand{\gca}{Geochim. Cosmochim. Acta}$
$\newcommand{\grl}{Geophys. Res. Lett.}$
$\newcommand{\icarus}{Icarus}$
$\newcommand{\ija}{Int. J. Astrobiol.}$
$\newcommand{\jatis}{J. Astron. Telesc. Instrum. Syst.}$
$\newcommand{\jcap}{J. Cosmol. Astropart. Phys.}$
$\newcommand{\jgr}{J. Geophys. Res.}$
$\newcommand{\jgrp}{J. Geophys. Res.: Planets}$
$\newcommand{\jqsrt}{J. Quant. Spectrosc. Radiat. Transf.}$
$\newcommand{\lrca}{Living Rev. Comput. Astrophys.}$
$\newcommand{\lrr}{Living Rev. Relativ.}$
$\newcommand{\lrsp}{Living Rev. Sol. Phys.}$
$\newcommand{\memsai}{Mem. Soc. Astron. Italiana}$
$\newcommand{\maps}{Meteorit. Planet. Sci.}$
$\newcommand{\mnras}{Mon. Not. R. Astron. Soc.}$
$\newcommand{\nastro}{Nat. Astron.}$
$\newcommand{\ncomms}{Nat. Commun.}$
$\newcommand{\ngeo}{Nat. Geosci.}$
$\newcommand{\nphys}{Nat. Phys.}$
$\newcommand{\na}{New Astron.}$
$\newcommand{\nar}{New Astron. Rev.}$
$\newcommand{\physrep}{Phys. Rep.}$
$\newcommand{\psj}{Planet. Sci. J.}$
$\newcommand{\planss}{Planet. Space Sci.}$
$\newcommand{\pnas}{Proc. Natl Acad. Sci. USA}$
$\newcommand{\procspie}{Proc. SPIE}$
$\newcommand{\spieproc}{Proc. SPIE}$
$\newcommand{\pasa}{Publ. Astron. Soc. Aust.}$
$\newcommand{\pasj}{Publ. Astron. Soc. Jpn}$
$\newcommand{\pasp}{Publ. Astron. Soc. Pac.}$
$\newcommand{\raa}{Res. Astron. Astrophys.}$
$\newcommand{\rmxaa}{Rev. Mexicana Astron. Astrofis.}$
$\newcommand{\sci}{Science}$
$\newcommand{\sciadv}{Sci. Adv.}$
$\newcommand{\solphys}{Sol. Phys.}$
$\newcommand{\sovast}{Soviet Astron.}$
$\newcommand{\ssr}{Space Sci. Rev.}$
$\newcommand{\uni}{Universe}$
$\newcommand{\apj}{Astrophys. J.}$
$\newcommand{\ao}{Appl. Opt.}$</div>



<div id="title">

# Direct imaging of AGN outflows and their origin with the 23 m Large Binocular Telescope

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.01840-b31b1b.svg)](https://arxiv.org/abs/2502.01840)<mark>Appeared on: 2025-02-05</mark> -  _26 Pages, Accepted for publication in Nature Astronomy_

</div>
<div id="authors">

J. W. Isbell, et al. -- incl., <mark>J.-U. Pott</mark>

</div>
<div id="abstract">

**Abstract:** Active galactic nuclei (AGNs) are a key component of galaxy evolution due to feedback on the host from its supermassive black hole. The morphology of warm, in- and outflowing dusty material can reveal the nature of the onset of feedback, AGN feeding, and the unified model of AGN. Here we use the Large Binocular Telescope Interferometer (LBTI) to image the dense, obscuring disk and extended dusty outflow region of NGC 1068.  In Fizeau imaging mode the LBTI synthesizes the equivalent resolution of a 22.8 m telescope.  The 8.7 $\mu$ m Fizeau images of NGC 1068 ${have an effective resolution of $47\times90$ mas ($3.3\times6.2$ pc)}$ in a 5" field of view after performing PSF deconvolution techniques described here.This is the only extragalactic source to be Fizeau imaged using the LBTI, and the images bridge the scales measured with the Very Large Telescope Interferometer (VLTI; 0.5-5 pc) and those of single telescopes such as JWST and Keck ( $>15$ pc). The images detect and spatially resolve the low surface brightness mid-infrared (MIR) features in the AGN disk/wind region that are over-resolved by the VLTI.  The images show strong correlation between MIR dust emission and near-infrared (NIR) emission of highly excited atomic lines observed by SINFONI.Such LBTI imaging is a precursor to infrared imaging using the upcoming generation of extremely large telescopes, with angular resolutions up to 6x better than JWST, the largest space telescope in orbit.

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.01840"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

326  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
